In [ ]:
# pip install geemap geopandas

In [ ]:
import geemap, ee

ee.Authenticate(auth_mode="notebook")
ee.Initialize(project="ee-mtpictd")

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=losWGZkYUKXrHxs0l9uLWmmMp4gxNpoinfkVtgLIq-I&tc=hD07pWbu6BY3IiHb4nOXenj3a3ZVB_UhGZb_ER6liBI&cc=sgM6sEHelyGbo9jfJqFc_yCSTOrli-XvVGekeIg0fIU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkX_FVsMa2cu2IYbf-MCrv7FhKfcAVD74YEgjd1cqcCRnFrgFye3Sg

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import geopandas as gpd
import pandas as pd
import json
import numpy as np
from glob import glob
from time import sleep
import re

In [ ]:
agro_dist_df = pd.read_csv("drive/MyDrive/dhruvi/data/distict_agroclimate_map.csv")

In [ ]:
agro_dist_df.head(2)

,Name,geometry,intersecting_zones,common_area,agroclimatic_zone
0,Bid,"POLYGON ((75.4811165057046 19.421690425589254,...",['Western Plateau and Hills Region'],[0.9788641089465046],Western Plateau and Hills Region
1,Diu,POLYGON ((70.89208173197272 20.707918187707488...,[],[],NaN


In [ ]:
agro_dist_df[agro_dist_df['Name'] == 'North 24 Parganas']

,Name,geometry,intersecting_zones,common_area,agroclimatic_zone
647,North 24 Parganas,MULTIPOLYGON (((88.33101503161164 22.772098428...,['Lower Gangetic Plain Region'],[0.5093258136146036],Lower Gangetic Plain Region


In [ ]:
agro_dist_df['agroclimatic_zone'].unique()

array(['Western Plateau and Hills Region', nan,
       'Middle Gangetic Plain Region', 'Eastern Himalayan Region',
       'Trans Gangetic Plain Region', 'Upper Gangetic Plain Region',
       'Western Himalayan Region', 'Eastern Plateau & Hills Region',
       'Central Plateau & Hills Region',
       'West Coast Plains & Ghat Region', 'Western Dry Region',
       'East Coast Plains & Hills Region',
       'Southern Plateau and Hills Region', 'Lower Gangetic Plain Region',
       'Island region'], dtype=object)

In [ ]:
# dist_list = list(agro_dist_df[agro_dist_df['agroclimatic_zone'].isin(['Eastern Plateau & Hills Region', #'Southern Plateau and Hills Region',
#                                                                       'Lower Gangetic Plain Region'])]['Name'])
# len(dist_list)

In [ ]:
dist_list = list(agro_dist_df[~agro_dist_df['agroclimatic_zone'].isin(['Island region', 'Eastern Plateau & Hills Region', 'Lower Gangetic Plain Region'])]['Name'])
len(dist_list)

591

In [ ]:
# ['cover_z' + str(i) for i in range(1,31)] + ['fhd_normal', 'cover', 'pai', 'pgap_theta']

In [ ]:
start_date = {'kharif': '2019-07-01', 'rabi': '2019-11-01', 'zaid': '2020-03-01'}
end_date = {'kharif': '2019-10-31', 'rabi': '2020-02-28', 'zaid': '2020-06-30'}

# start_date = {'year': '2019-07-01'}
# end_date = {'year': '2020-06-30'}

india_boundary = ee.FeatureCollection("projects/ee-indiasat/assets/india_district_boundaries")
gedi_chm_bands = ['rh10', 'rh25', 'rh50', 'rh75', 'rh90', 'rh95', 'rh98', 'rh100', 'delta_time']
gedi_cc_bands = ['cover', 'fhd_normal', 'pai', 'pgap_theta', 'delta_time']
s1_bands = ['VV', 'VH', 'angle']
s2_bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11','B12']
s2_bands_min = [band + '_min' for band in s2_bands]
s2_bands_max = [band + '_max' for band in s2_bands]

In [ ]:
def gedi_chm_mask(im):
  dem = 'digital_elevation_model'
  srtm = 'digital_elevation_model_srtm'
  diff = im.select(dem).subtract(im.select(srtm)).abs();
  dem_mask = diff.lte(75)
  return im.updateMask(im.select('quality_flag').eq(1)).updateMask(im.select('degrade_flag').eq(0)).updateMask(
      im.select('rh25').gte(0)).updateMask(im.select('sensitivity').gte(0.98)).updateMask(
          im.select('num_detectedmodes').neq(0)).updateMask(dem_mask)

def get_gedi_chm_image(aoi, start_date, end_date):
  return ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filterDate(
      start_date , end_date).filterBounds(aoi).map(gedi_chm_mask).select(gedi_chm_bands).median().clip(aoi)

def gedi_cc_mask(im):
  return im.updateMask(im.select('l2b_quality_flag').eq(1)).updateMask(
      im.select('degrade_flag').eq(0)).updateMask(im.select('sensitivity').gte(0.98))

def get_gedi_cc_image(aoi, start_date, end_date):
  return ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY').filterDate(
      start_date , end_date).filterBounds(aoi).map(gedi_cc_mask).select(gedi_cc_bands).median().clip(aoi)

def get_s1_image(aoi, start_date, end_date):
  # s1_bands_season = [band + '_' + season for band in s1_bands]
  return ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(start_date , end_date).filterBounds(aoi).filter(
      ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(
          ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(
              ee.Filter.eq('instrumentMode', 'IW')).select(s1_bands).median().clip(aoi)

def s2_mask(image):
  """
  Getting a cloud-free Sentinel-2 imagery.
  """
  quality_band = image.select('QA60')
  # Using the bit mask for clouds (bit 10) and cirrus clouds (bit 11) respectively.
  cloudmask = 1 << 10
  cirrusmask = 1 << 11
  # Both flags should be set to zero, indicating clear conditions of sky.
  mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
  return image.updateMask(mask)

def get_s2_image(aoi, start_date, end_date):
  # s2_bands_season = [band + '_med_' + season for band in s2_bands]
  return ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(
      start_date , end_date).filterBounds(aoi).filter(
          ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 1)).sort('CLOUD_COVER').map(
              s2_mask).select(s2_bands).median().divide(10000).clip(aoi)

def get_s2_image_min(aoi, start_date, end_date):
  return ee.ImageCollection('COPERNICUS/S2').filterDate(
      start_date , end_date).filterBounds(aoi).filter(
          ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 1)).sort('CLOUD_COVER').map(
              s2_mask).select(s2_bands).min().divide(10000).clip(aoi).rename(s2_bands_min)

def get_s2_image_max(aoi, start_date, end_date):
  return ee.ImageCollection('COPERNICUS/S2').filterDate(
      start_date , end_date).filterBounds(aoi).filter(
          ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 1)).sort('CLOUD_COVER').map(
              s2_mask).select(s2_bands).max().divide(10000).clip(aoi).rename(s2_bands_max)

In [ ]:
def save_satellite_image(satellite_image, aoi, img_name, out_path):
  task = ee.batch.Export.image.toDrive(
      image = satellite_image.clip(aoi),
      description = img_name,
      maxPixels = 1e20,
      scale = 25,
      folder = out_path,
      fileNamePrefix = img_name,
      region = aoi.geometry().bounds()
      )
  task.start()
  return task

def save_data_csv(data_points, aoi, img_name, out_path):
  task = ee.batch.Export.table.toDrive(
      collection = data_points,
      description = img_name,
      folder = out_path,
      fileNamePrefix = img_name,
      fileFormat = 'CSV'
      )
  task.start()
  return task

In [ ]:
GEDI_DATA = 'CHM'

if GEDI_DATA == 'CHM':
  get_gedi_image = get_gedi_chm_image
  gedi_bands = gedi_chm_bands
  out_path = 'gedi_chm_season_data_2019'
else:
  get_gedi_image = get_gedi_cc_image
  gedi_bands = gedi_cc_bands
  out_path = 'gedi_cc_season_data_2021'

In [ ]:
s1_task = {'kharif': dict(), 'rabi': dict(), 'zaid': dict()}
s2_task = {'kharif': dict(), 'rabi': dict(), 'zaid': dict()}
# s1_task = {'year': dict()}
# s2_task = {'year': dict()}

# s2_task_min = dict()
# s2_task_max = dict()
for i, dist in enumerate(dist_list[508:]):
  print(i, dist)
  try:
    img_name = re.sub("[^A-Za-z0-9]", "", dist)
    aoi = india_boundary.filter(ee.Filter.eq('Name', dist))
    tree_cover = ee.ImageCollection("ESA/WorldCover/v100").first().clip(aoi)
    # tree_cover = tree_cover.resample('bilinear').reproject(
    #   crs = tree_cover.projection().crs(),
    #   # "crs": 'EPSG:4326',
    #   scale = 25
    # )
    tree_cover = tree_cover.updateMask(tree_cover.eq(10))
    dw_tree_cover = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate(min(start_date.values()), max(end_date.values())).filterBounds(
        aoi).select('label').mode().clip(aoi.geometry());
    dw_tree_cover = dw_tree_cover.updateMask(dw_tree_cover.eq(1))
    tree_cover = tree_cover.updateMask(dw_tree_cover)
    # tree_cover = tree_cover.reproject(crs='EPSG:4326', scale=25)

    gedi_tree_data = get_gedi_image(aoi, min(start_date.values()), max(end_date.values()))

    # s1_data = get_s1_image(aoi)
    # s2_data = get_s2_image(aoi)
    # s2_data_min = get_s2_image_min(aoi)
    # s2_data_max = get_s2_image_max(aoi)
  #   # all_data = s1_data.addBands(s2_data).addBands(gedi_tree_data).updateMask(tree_cover).updateMask(
  #   #     gedi_tree_data.select('rh95')).updateMask(s1_data.select('VV')).updateMask(s2_data.select('B3'))

  except Exception as exp:
    print("Tree Cover Data Acquisition Error: ", exp)

  for season in start_date.keys():
    try:
      # gedi_tree_data = get_gedi_image(aoi, start_date[season], end_date[season])
      s1_data = get_s1_image(aoi, start_date[season], end_date[season])
      s1_tree_data = s1_data.addBands(gedi_tree_data).updateMask(tree_cover).updateMask(
          gedi_tree_data.select(gedi_bands[0])).select(s1_bands + gedi_bands)
      s1_data_points = s1_tree_data.sample(
          region = aoi,
          scale = 25,
          factor = 1,
          # "numPixels": 100,
          tileScale = 10,
          geometries = True
      )
      task = save_data_csv(s1_data_points, aoi, img_name + "_s1_" + season, out_path)
      s1_task[season][dist] = task

    except Exception as exp:
      print("S1 Error occured: ", season, exp)

    try:
      s2_data = get_s2_image(aoi, start_date[season], end_date[season])
      s2_tree_data = s2_data.addBands(gedi_tree_data).updateMask(tree_cover).updateMask(
          gedi_tree_data.select(gedi_bands[0])).select(s2_bands)
      s2_data_points = s2_tree_data.sample(
          region = aoi,
          scale = 25,
          factor = 1,
          # "numPixels": 100,
          tileScale = 10,
          geometries = True
      )
      task = save_data_csv(s2_data_points, aoi, img_name + "_s2_" + season, out_path)
      s2_task[season][dist] = task

      # s2_tree_data = s2_data_max.addBands(gedi_tree_data).updateMask(tree_cover).updateMask(
      #     gedi_tree_data.select(gedi_bands[0])).select(s2_bands_max)
      # s2_data_points = s2_tree_data.sample(
      #     region = aoi,
      #     scale = 25,
      #     factor = 1,
      #     # "numPixels": 100,
      #     tileScale = 10,
      #     geometries = True
      # )
      # task = save_data_csv(s2_data_points, aoi, img_name + "_s2_max", out_path)
      # s2_task_max[dist] = task

    except Exception as exp:
      print("S2 Error occured: ", season, exp)

0 Saharanpur
1 Samastipur
2 Sheikhpura
3 Sindhudurg
4 Sipahijala
5 Srikakulam
6 Tamenglong
7 Tarn Taran
8 Thiruvarur
9 Uttarkashi
10 Viluppuram
11 West Nimar
12 West Siang
13 Bulandshahr
14 Chikmagalur
15 Chitradurga
16 East Kameng
17 East Sikkim
18 Farrukhabad
19 Gandhinagar
20 Garhchiroli
21 Gir Somnath
22 Hanumangarh
23 Hoshangabad
24 Imphal East
25 Imphal West
26 Krishnagiri
27 Kurukshetra
28 Maharajganj
29 Mahbubnagar
30 Muzaffarpur
31 Narsimhapur
S1 Error occured:  kharif Too many tasks already in the queue (3000). Please wait for some of them to complete.
S2 Error occured:  kharif Too many tasks already in the queue (3000). Please wait for some of them to complete.
S1 Error occured:  rabi Too many tasks already in the queue (3000). Please wait for some of them to complete.
S2 Error occured:  rabi Too many tasks already in the queue (3000). Please wait for some of them to complete.
S1 Error occured:  zaid Too many tasks already in the queue (3000). Please wait for some of them to

KeyboardInterrupt: 

In [ ]:
dist_list.index("Narsimhapur")

508

In [ ]:
for i, dist in enumerate(dist_list[200:207]):
  for s in ['kharif', 'rabi', 'zaid']:
    s1_state = s1_task[s][dist].status()['state']
    s2_state = s2_task[s][dist].status()['state']
    print(dist, s, s1_state, s2_state)
    if s1_state == "FAILED":
      print(s1_task[s][dist].status())
    if s2_state == "FAILED":
      print(s2_task[s][dist].status())

#   sleep(5)

Bhiwani kharif COMPLETED COMPLETED
Bhiwani rabi COMPLETED COMPLETED
Bhiwani zaid COMPLETED COMPLETED
Bhojpur kharif COMPLETED COMPLETED
Bhojpur rabi COMPLETED COMPLETED
Bhojpur zaid COMPLETED COMPLETED
Bijapur kharif COMPLETED FAILED
{'state': 'FAILED', 'description': 'Bijapur_s2_kharif', 'creation_timestamp_ms': 1705138647953, 'update_timestamp_ms': 1705146450754, 'start_timestamp_ms': 1705146449209, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'error_message': 'Image.divide: If one image has no bands, the other must also have no bands. Got 0 and 1.', 'id': 'VGNR7IZIXCYDB3GR3MT5YCNI', 'name': 'projects/ee-mtpictd/operations/VGNR7IZIXCYDB3GR3MT5YCNI'}
Bijapur rabi COMPLETED COMPLETED
Bijapur zaid COMPLETED COMPLETED
Bijapur kharif COMPLETED FAILED
{'state': 'FAILED', 'description': 'Bijapur_s2_kharif', 'creation_timestamp_ms': 1705138647953, 'update_timestamp_ms': 1705146450754, 'start_timestamp_ms': 1705146449209, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'error_message': 'Image.